In [4]:
from langchain.agents import tool
import requests
import json
from dotenv import load_dotenv
import os

load_dotenv()

@tool
def get_weather(city: str) -> str:
  """Get the weather for a given city."""
  return f"It's always sunny in {city}!"

def get_smiles_exchange_rates(from_currency: str, to_currency: str):
    """Get exchange rate for a given currency pair."""
    smiles_exchange_api = os.getenv("SMILES_EXCHANGE_API")
    if not smiles_exchange_api:
        raise ValueError("SMILES_EXCHANGE_API environment variable is not set.")
    response = requests.get(smiles_exchange_api)
    if response.status_code != 200:
        raise Exception(f"Error: {response.status_code}")
    response_json = response.json()
    rates = json.loads(response_json["Rates"])
    currency = rates["ALL_ALL_ALL"]["Currency"]
    print(currency)

    exchange_rate_data = []
    for value in currency:
        data = currency[value]
        exchange_rate_data.append(
            {
                "sender_unit": float(1),
                "sender_currency": data["From"],
                "receiver_unit": float(data["SellingRate"]),
                "receiver_currency": data["To"],
            }
        )
    for rate in exchange_rate_data:
      if rate["sender_currency"] == from_currency and rate["receiver_currency"] == to_currency:
        return f"%s" % rate["receiver_unit"]
    return f"Sorry we don't have exchange rate for {from_currency} to {to_currency}."

from langchain_ollama import ChatOllama
llm = ChatOllama(model="mistral", temperature=0.3)

from langgraph.prebuilt import create_react_agent, ToolNode
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver

agent_node = create_react_agent(llm, [get_weather, get_smiles_exchange_rates])
tool_node = ToolNode([get_weather, get_smiles_exchange_rates])

graph = StateGraph(MessagesState)
graph.add_node("agent", agent_node)
graph.add_node("tools", tool_node)

graph.add_edge(START, "agent")
def agent_router(state):
  # state["messages"] is a list of messages
  messages = state["messages"]
  print(f"Messages: {messages}")
  last = messages[-1]
  # Tool calls are usually FunctionMessage or have tool_call/tool_calls attribute
  if getattr(last, "tool_call", None) or getattr(last, "tool_calls", None):
      return "tools"
  # If it's an AIMessage and not a tool call, it's final
  return END

graph.add_conditional_edges("agent", path=agent_router)
graph.add_edge("tools", "agent")
app = graph.compile(checkpointer=MemorySaver())

from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig
msg = [HumanMessage(content="Ram lives and works in Singapore. He wants to send money to his family in Indonesia. Find exchange rate.")]
config: RunnableConfig = {"configurable": {"thread_id": "thread_1"}}

response = app.invoke({"messages": msg}, config=config)
print(response["messages"][-1].content)

{'Currency_USD_JPY': {'From': 'USD', 'To': 'JPY', 'BuyingRate': None, 'SellingRate': 140.59, 'BookRate': 142.73, 'AverageRate': None, 'TTMRate': None}, 'Currency_JPY_USD': {'From': 'JPY', 'To': 'USD', 'BuyingRate': None, 'SellingRate': 0.006957, 'BookRate': 0.007006, 'AverageRate': None, 'TTMRate': None}, 'Currency_JPY_PHP': {'From': 'JPY', 'To': 'PHP', 'BuyingRate': None, 'SellingRate': 0.389, 'BookRate': 0.3882, 'AverageRate': None, 'TTMRate': None}, 'Currency_PHP_JPY': {'From': 'PHP', 'To': 'JPY', 'BuyingRate': None, 'SellingRate': 2.5374, 'BookRate': 2.576, 'AverageRate': None, 'TTMRate': None}, 'Currency_JPY_IDR': {'From': 'JPY', 'To': 'IDR', 'BuyingRate': None, 'SellingRate': 113.4, 'BookRate': 113.7876, 'AverageRate': None, 'TTMRate': None}, 'Currency_IDR_JPY': {'From': 'IDR', 'To': 'JPY', 'BuyingRate': None, 'SellingRate': 0.008656, 'BookRate': 0.008788, 'AverageRate': None, 'TTMRate': None}, 'Currency_JPY_VND': {'From': 'JPY', 'To': 'VND', 'BuyingRate': None, 'SellingRate': 18